In [1]:
# =========================
# [CELDA 1] Setup + Paths + Config
# =========================
import os
from pathlib import Path
import numpy as np
import pandas as pd

# Reproducibilidad (para sampling/plots si lo usas)
SEED = 42
np.random.seed(SEED)

# --- Proyecto ---
PROJECT_DIR = Path("/workspace/TFM_education_ai_analytics")

# --- Entradas base (ya generadas por tus pipelines) ---
PROCESSED_DIR = PROJECT_DIR / "data/2_processed"          # students/interactions/assessments por split
FEATURES_DIR  = PROJECT_DIR / "data/3_features"           # engineered_features.csv + target.csv por split (FE1)
EMB_DIR       = PROJECT_DIR / "data/4_embeddings"         # segmentation_gmm_ae.csv por split

# --- Salidas para el Transformer (nuevo) ---
OUT_DIR = PROJECT_DIR / "data/6_transformer_features"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Splits
SPLITS = ["training", "validation", "test"]

# Ventanas de semanas (como en el paper, adaptado a tu caso)
WEEKS_LIST = [12, 16, 20, 24]

print("✅ Paths OK")
print("PROJECT_DIR:", PROJECT_DIR)
print("OUT_DIR:", OUT_DIR)
print("WEEKS_LIST:", WEEKS_LIST)


✅ Paths OK
PROJECT_DIR: /workspace/TFM_education_ai_analytics
OUT_DIR: /workspace/TFM_education_ai_analytics/data/6_transformer_features
WEEKS_LIST: [12, 16, 20, 24]


In [2]:
# =========================
# [CELDA 2] Sanity check + carga rápida (TRAIN)
# =========================
from pathlib import Path

def check_exists(path: Path, label: str):
    if not path.exists():
        raise FileNotFoundError(f"❌ No existe {label}: {path}")
    return True

# --- Archivos esperados por split ---
expected = {
    "processed_students": lambda split: PROCESSED_DIR / split / "students.csv",
    "processed_interactions": lambda split: PROCESSED_DIR / split / "interactions.csv",
    "processed_assessments": lambda split: PROCESSED_DIR / split / "assessments.csv",
    "target": lambda split: FEATURES_DIR / split / "target.csv",
    "engineered": lambda split: FEATURES_DIR / split / "engineered_features.csv",
"segmentation": lambda split: (PROJECT_DIR / "data/5_students_segmented") / split / "segmentation_gmm_ae.csv",
}

# 1) Comprobar existencia
for split in SPLITS:
    print(f"\n🔎 Checking split: {split}")
    for name, fn in expected.items():
        p = fn(split)
        check_exists(p, f"{name} ({split})")
        print(f"  ✅ {name}: {p.name}")

# 2) Cargar TRAIN para inspección rápida
train_students = pd.read_csv(expected["processed_students"]("training"))
train_interactions = pd.read_csv(expected["processed_interactions"]("training"))
train_target = pd.read_csv(expected["target"]("training"), index_col=0)
train_seg = pd.read_csv(expected["segmentation"]("training"), index_col=0)

print("\n📌 TRAIN shapes:")
print("students:", train_students.shape)
print("interactions:", train_interactions.shape)
print("target:", train_target.shape)
print("segmentation:", train_seg.shape)

print("\n📌 TRAIN columns (heads):")
display(train_students.head(3))
display(train_interactions.head(3))
display(train_target.head(3))
display(train_seg.head(3))



🔎 Checking split: training
  ✅ processed_students: students.csv
  ✅ processed_interactions: interactions.csv
  ✅ processed_assessments: assessments.csv
  ✅ target: target.csv


FileNotFoundError: ❌ No existe engineered (training): /workspace/TFM_education_ai_analytics/data/3_features/training/engineered_features.csv

In [ ]:
# =========================
# [CELDA INDEPENDIENTE] Estáticas desde students.csv (safe) + schema TRAIN + scaler global + merge con FE2 + guardado
# =========================
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# ---- CONFIG ----
W = 12  # cambia si quieres otra ventana

# Columnas estáticas "safe" (inicio de curso)
STATIC_CATEGORICAL = ["gender", "region", "highest_education", "imd_band", "age_band", "disability"]
STATIC_NUMERIC = ["num_of_prev_attempts", "studied_credits", "date_registration", "module_presentation_length"]

# Nunca usar (leakage/target)
EXCLUDE_COLS = ["final_result", "date_unregistration"]

# Paths FE2 (ya generados previamente)
OUT_DIR = PROJECT_DIR / "data/6_transformer_features"  # asegúrate que PROJECT_DIR existe
SPLITS = ["training", "validation", "test"]


# ---- HELPERS ----
def _require(name: str):
    if name not in globals():
        raise NameError(
            f"Falta la variable '{name}' en el notebook. "
            f"Asegúrate de haber cargado los dataframes de students y FE2 antes de ejecutar esta celda."
        )

def build_unique_id_from_students(df_students: pd.DataFrame) -> pd.Series:
    """
    Construye unique_id = id_student_code_module_code_presentation
    (basado en tu convención del proyecto)
    """
    return (
        df_students["id_student"].astype(str)
        + "_"
        + df_students["code_module"].astype(str)
        + "_"
        + df_students["code_presentation"].astype(str)
    )

def build_static_from_students(df_students: pd.DataFrame) -> pd.DataFrame:
    """
    Devuelve X_static con index=unique_id.
    - OHE para categóricas
    - numéricas como float (sin escalar aquí)
    - excluye columnas peligrosas
    """
    df = df_students.copy()

    # Excluir leakage/target-like si existieran
    for c in EXCLUDE_COLS:
        if c in df.columns:
            df = df.drop(columns=[c])

    # unique_id
    if "unique_id" not in df.columns:
        df["unique_id"] = build_unique_id_from_students(df)

    df = df.set_index("unique_id")

    # Numéricas
    num_cols = [c for c in STATIC_NUMERIC if c in df.columns]
    for c in num_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0.0).astype(float)

    # Categóricas
    cat_cols = [c for c in STATIC_CATEGORICAL if c in df.columns]
    X_cat = pd.get_dummies(
        df[cat_cols].fillna("UNKNOWN").astype(str),
        prefix=cat_cols,
        drop_first=False,
    )

    # Unir
    X_num = df[num_cols].copy()
    X = pd.concat([X_num, X_cat], axis=1)

    X = X.replace([np.inf, -np.inf], np.nan).fillna(0.0)
    X.index.name = "unique_id"
    return X


def apply_static_schema_and_scaler(
    df_students: pd.DataFrame,
    schema_cols: list[str],
    scaler: StandardScaler,
    numeric_cols_present_in_schema: list[str],
) -> pd.DataFrame:
    """
    Construye estáticas, las reindexa a schema TRAIN, y aplica scaler solo a numéricas.
    """
    X = build_static_from_students(df_students)
    X = X.reindex(columns=schema_cols, fill_value=0.0)

    cols_to_scale = [c for c in numeric_cols_present_in_schema if c in X.columns]
    if cols_to_scale:
        X[cols_to_scale] = scaler.transform(X[cols_to_scale])

    X = X.replace([np.inf, -np.inf], np.nan).fillna(0.0)
    return X


# ---- REQUIRE INPUTS ----
# Necesitas estos dataframes ya cargados en tu notebook:
# - train_students, val_students, test_students  (desde data/2_processed/<split>/students.csv)
# - (opción A) X_train_w12, X_val_w12, X_test_w12 (FE2 en memoria)
#   o (opción B) wXX_X_seq.csv guardados en disco
for v in ["train_students", "val_students", "test_students"]:
    _require(v)

# Detectar si tienes FE2 en memoria; si no, lo cargamos desde disco
have_in_memory = all([f"X_{sp}_w{W}" in globals() for sp in ["train", "val", "test"]])

def load_fe2_seq(split: str) -> pd.DataFrame:
    path = OUT_DIR / split / f"w{W:02d}_X_seq.csv"
    if not path.exists():
        raise FileNotFoundError(f"No existe FE2 secuencias: {path}")
    return pd.read_csv(path, index_col=0)

if have_in_memory:
    X_train_seq = globals()[f"X_train_w{W}"]
    X_val_seq   = globals()[f"X_val_w{W}"]
    X_test_seq  = globals()[f"X_test_w{W}"]
else:
    X_train_seq = load_fe2_seq("training")
    X_val_seq   = load_fe2_seq("validation")
    X_test_seq  = load_fe2_seq("test")


# ---- BUILD STATIC (TRAIN schema + scaler) ----
X_static_train = build_static_from_students(train_students)

static_schema = X_static_train.columns.tolist()
numeric_cols_in_schema = [c for c in STATIC_NUMERIC if c in X_static_train.columns]

scaler_static = StandardScaler()
if numeric_cols_in_schema:
    X_static_train[numeric_cols_in_schema] = scaler_static.fit_transform(X_static_train[numeric_cols_in_schema])

X_static_val = apply_static_schema_and_scaler(
    val_students,
    schema_cols=static_schema,
    scaler=scaler_static,
    numeric_cols_present_in_schema=numeric_cols_in_schema,
)
X_static_test = apply_static_schema_and_scaler(
    test_students,
    schema_cols=static_schema,
    scaler=scaler_static,
    numeric_cols_present_in_schema=numeric_cols_in_schema,
)

print("✅ static shapes:", X_static_train.shape, X_static_val.shape, X_static_test.shape)


# ---- MERGE STATIC + FE2 SEQ ----
X_train_hybrid = X_train_seq.join(X_static_train, how="left").fillna(0.0)
X_val_hybrid   = X_val_seq.join(X_static_val,   how="left").fillna(0.0)
X_test_hybrid  = X_test_seq.join(X_static_test, how="left").fillna(0.0)

print("✅ hybrid shapes:", X_train_hybrid.shape, X_val_hybrid.shape, X_test_hybrid.shape)


# ---- SAVE ----
def save_hybrid(split: str, df: pd.DataFrame):
    out_dir = OUT_DIR / split
    os.makedirs(out_dir, exist_ok=True)
    out_path = out_dir / f"w{W:02d}_X_hybrid.csv"
    df.to_csv(out_path)
    print("💾 guardado:", out_path)

save_hybrid("training", X_train_hybrid)
save_hybrid("validation", X_val_hybrid)
save_hybrid("test", X_test_hybrid)

# Preview
display(X_train_hybrid.head(3))



NameError: name 'PROJECT_DIR' is not defined

In [ ]:
# =========================
# [CELDA 5] Prestart + secuencia semanal + robust scaling (log1p + clip) + normalización por curso (fit TRAIN)
# =========================
import numpy as np
import pandas as pd

EPS = 1e-6

def _ensure_numeric(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    out = df.copy()
    for c in cols:
        if c in out.columns:
            out[c] = pd.to_numeric(out[c], errors="coerce").fillna(0.0)
    return out

def add_prestart_features(
    df_interactions: pd.DataFrame,
    index_uids: pd.Index,
    clicks_col: str = "sum_click",
    day_col: str = "date",
) -> pd.DataFrame:
    df = df_interactions.copy()
    df = add_unique_id(df)

    df[clicks_col] = pd.to_numeric(df[clicks_col], errors="coerce").fillna(0).astype(float)
    df[day_col] = pd.to_numeric(df[day_col], errors="coerce").astype(float)

    pre = df[df[day_col] < 0].copy()

    out = pd.DataFrame(index=index_uids)
    if pre.empty:
        out["prestart_clicks_total"] = 0.0
        out["prestart_active_days"] = 0
        out["prestart_active_weeks"] = 0
        out["prestart_earliest_day"] = 0.0
        out["investigated_platform"] = 0
        return out

    pre["week"] = np.floor(pre[day_col] / 7.0).astype(int)

    agg = pre.groupby("unique_id").agg(
        prestart_clicks_total=(clicks_col, "sum"),
        prestart_active_days=(day_col, lambda s: int(pd.Series(s.dropna().astype(int)).nunique())),
        prestart_active_weeks=("week", lambda s: int(pd.Series(s.dropna().astype(int)).nunique())),
        prestart_earliest_day=(day_col, "min"),
    )

    agg = agg.reindex(index=index_uids, fill_value=0.0)
    agg["prestart_active_days"] = agg["prestart_active_days"].astype(int)
    agg["prestart_active_weeks"] = agg["prestart_active_weeks"].astype(int)
    agg["prestart_clicks_total"] = agg["prestart_clicks_total"].astype(float)
    agg["prestart_earliest_day"] = agg["prestart_earliest_day"].astype(float)

    agg["investigated_platform"] = (agg["prestart_clicks_total"] > 0).astype(int)
    return agg


def build_weekly_sequence_features(
    df_interactions: pd.DataFrame,
    index_uids: pd.Index,
    weeks: int,
    activity_col: str = "activity_type",
    clicks_col: str = "sum_click",
    day_col: str = "date",
) -> pd.DataFrame:
    df = df_interactions.copy()
    df = add_unique_id(df)

    df[clicks_col] = pd.to_numeric(df[clicks_col], errors="coerce").fillna(0).astype(float)
    df[day_col] = pd.to_numeric(df[day_col], errors="coerce").astype(float)

    df["week"] = np.floor(df[day_col] / 7.0).astype(int)
    df = df[(df["week"] >= 0) & (df["week"] < weeks)].copy()

    g = df.groupby(["unique_id", "week", activity_col])[clicks_col].sum().reset_index()

    pivot = g.pivot_table(
        index="unique_id",
        columns=["week", activity_col],
        values=clicks_col,
        aggfunc="sum",
        fill_value=0.0,
    )
    pivot.columns = [f"clicks_{str(act).lower()}_w{int(w):02d}" for (w, act) in pivot.columns]
    pivot = pivot.sort_index(axis=1)

    weekly_total = df.groupby(["unique_id", "week"])[clicks_col].sum().unstack(fill_value=0.0)
    for w in range(weeks):
        if w not in weekly_total.columns:
            weekly_total[w] = 0.0
    weekly_total = weekly_total[sorted(weekly_total.columns)]
    weekly_total.columns = [f"total_clicks_w{int(w):02d}" for w in weekly_total.columns]

    active_weeks = (weekly_total.values > 0).sum(axis=1)
    active_weeks = pd.Series(active_weeks, index=weekly_total.index, name="active_weeks_uptoW")

    total_sum = weekly_total.sum(axis=1).astype(float)
    early_weeks = list(range(min(4, weeks)))
    early_sum = weekly_total[[f"total_clicks_w{w:02d}" for w in early_weeks]].sum(axis=1).astype(float)
    early_ratio = (early_sum / (total_sum + EPS)).fillna(0.0).rename("early_ratio_uptoW")

    late_weeks = list(range(max(0, weeks - 4), weeks))
    late_sum = weekly_total[[f"total_clicks_w{w:02d}" for w in late_weeks]].sum(axis=1).astype(float)
    late_ratio = (late_sum / (total_sum + EPS)).fillna(0.0).rename("late_ratio_uptoW")

    out = pivot.join(weekly_total, how="outer").join(active_weeks).join(early_ratio).join(late_ratio).fillna(0.0)
    out = out.reindex(index=index_uids, fill_value=0.0)
    out.index.name = "unique_id"
    return out


def clip_and_log1p(df: pd.DataFrame, p_clip: float = 0.995) -> pd.DataFrame:
    """
    1) Clip por percentil (solo sobre columnas de clicks)
    2) log1p
    """
    out = df.copy()
    click_cols = [c for c in out.columns if c.startswith("clicks_") or c.startswith("total_clicks_")]
    if not click_cols:
        return out

    # clip global (más simple y robusto); evita outliers 500000
    vals = out[click_cols].values.astype(float).ravel()
    clip_val = float(np.quantile(vals, p_clip))
    if clip_val <= 0:
        clip_val = 0.0

    out[click_cols] = np.clip(out[click_cols].astype(float), 0.0, clip_val)
    out[click_cols] = np.log1p(out[click_cols].astype(float))
    return out

def fit_course_stats(
    X_seq: pd.DataFrame,
    course_series: pd.Series,
) -> dict:
    """
    Fit medias/std por curso SOLO para columnas de clicks.
    """
    X_seq = X_seq.copy()
    course_series = course_series.reindex(X_seq.index)
    course_series = course_series.fillna("UNKNOWN").astype(str)

    click_cols = [c for c in X_seq.columns if c.startswith("clicks_") or c.startswith("total_clicks_")]
    stats = {}

    for course, idxs in course_series.groupby(course_series).groups.items():
        block = X_seq.loc[idxs, click_cols]
        mu = block.mean(axis=0)
        sd = block.std(axis=0).replace(0, 1.0)
        stats[str(course)] = {c: {"mean": float(mu[c]), "std": float(sd[c])} for c in click_cols}

    return stats


def apply_course_norm(
    X_seq: pd.DataFrame,
    course_series: pd.Series,
    course_stats: dict,
) -> pd.DataFrame:
    """
    Aplica z-score por curso SOLO a columnas de clicks.
    Ratios (0..1) y features prestart no se normalizan por curso.
    """
    out = X_seq.copy()
    course_series = course_series.reindex(out.index)
    course_series = course_series.fillna("UNKNOWN").astype(str)

    click_cols = [c for c in out.columns if c.startswith("clicks_") or c.startswith("total_clicks_")]
    if not click_cols:
        return out

    # fallback global
    global_mu = out[click_cols].mean(axis=0)
    global_sd = out[click_cols].std(axis=0).replace(0, 1.0)

    for course, idxs in course_series.groupby(course_series).groups.items():
        ckey = str(course)
        if ckey in course_stats:
            mu = pd.Series({col: course_stats[ckey][col]["mean"] for col in click_cols})
            sd = pd.Series({col: course_stats[ckey][col]["std"] for col in click_cols}).replace(0, 1.0)
        else:
            mu, sd = global_mu, global_sd

        out.loc[idxs, click_cols] = (out.loc[idxs, click_cols] - mu) / sd

    out[click_cols] = out[click_cols].replace([np.inf, -np.inf], np.nan).fillna(0.0)
    return out




def build_transformer_block_for_split(
    split_students_al: pd.DataFrame,     # index=unique_id, con code_module/code_presentation
    split_interactions: pd.DataFrame,    # raw interactions del split
    weeks: int,
    course_stats: dict | None = None,    # si None => fit en este split (solo TRAIN)
    p_clip: float = 0.995,
):
    """
    Devuelve:
      - X_seq_norm: secuencia semanal (0..W-1) + features agregadas + prestart, ya normalizado
      - course_stats (si se fittea en este split)
    """
    # course key (module_presentation)
    course = (
        split_students_al["code_module"].astype(str)
        + "_"
        + split_students_al["code_presentation"].astype(str)
    )

    # 1) secuencia + agregados
    X_seq = build_weekly_sequence_features(split_interactions, split_students_al.index, weeks=weeks)

    # 2) prestart
    X_pre = add_prestart_features(split_interactions, split_students_al.index)

    # ratio prestart vs actividad en ventana (sobre TOTAL semanal)
    total_uptoW = X_seq[[c for c in X_seq.columns if c.startswith("total_clicks_w")]].sum(axis=1).astype(float)
    X_pre["prestart_ratio_vs_uptoW"] = (X_pre["prestart_clicks_total"] / (total_uptoW + EPS)).fillna(0.0)

    # 3) unir
    X = pd.concat([X_seq, X_pre], axis=1)

    # 4) robust anti-outliers en clicks (clip + log1p)
    X = clip_and_log1p(X, p_clip=p_clip)

    # 5) normalización por curso (fit solo en TRAIN)
    if course_stats is None:
        course_stats = fit_course_stats(X, course)
    X = apply_course_norm(X, course, course_stats)

    return X, course_stats


# --------- EJEMPLO (TRAIN, W=12) ---------
W = 12
X_train_w12, course_stats_train = build_transformer_block_for_split(
    split_students_al=train_students_al,
    split_interactions=train_interactions,
    weeks=W,
    course_stats=None,      # FIT aquí (solo TRAIN)
    p_clip=0.995,
)

print("✅ X_train_w12 (seq+prestart+norm):", X_train_w12.shape)
display(X_train_w12.head(3))

# Guardamos stats en memoria para aplicar en val/test luego
COURSE_STATS = course_stats_train


✅ X_train_w12 (seq+prestart+norm): (22785, 236)


,clicks_dataplus_w00,clicks_dataplus_w01,clicks_dataplus_w02,clicks_dataplus_w03,clicks_dataplus_w04,clicks_dataplus_w05,clicks_dataplus_w06,clicks_dataplus_w07,clicks_dataplus_w08,clicks_dataplus_w09,...,total_clicks_w11,active_weeks_uptoW,early_ratio_uptoW,late_ratio_uptoW,prestart_clicks_total,prestart_active_days,prestart_active_weeks,prestart_earliest_day,investigated_platform,prestart_ratio_vs_uptoW
unique_id,,,,,,,,,,,,,,,,,,,,,
11391_AAA_2013J,-0.144014,-0.132509,-0.061898,-0.129121,-0.177694,-0.218672,-0.140828,-0.113607,-0.104414,-0.087706,...,1.167762,10.0,0.622177,0.114990,98.0,1,1,-5.0,1,0.201232
28400_AAA_2013J,-0.144014,-0.132509,-0.061898,-0.129121,-0.177694,-0.218672,-0.140828,-0.113607,-0.104414,-0.087706,...,0.551847,11.0,0.612431,0.170018,215.0,7,2,-10.0,1,0.393053
32885_AAA_2013J,-0.144014,-0.132509,-0.061898,-0.129121,-0.177694,-0.218672,-0.140828,-0.113607,-0.104414,-0.087706,...,1.436596,10.0,0.713911,0.196850,295.0,8,2,-10.0,1,0.774278


In [ ]:
# =========================
# [CELDA 7] Generar y Guardar VALIDATION y TEST (Usando stats de Train)
# =========================
# IMPORTANTE: Usamos 'COURSE_STATS' calculado en la celda anterior (Train)
# para que la normalización sea consistente y sin data leakage.

splits_to_process = [
    ("validation", val_students_al, val_interactions_al, val_target_al),
    ("test", test_students_al, test_interactions_al, test_target_al)
]

for split_name, df_stud, df_inter, df_tgt in splits_to_process:
    print(f"\n🚀 Procesando split: {split_name.upper()} (W={W}) ...")
    
    # Notar que pasamos course_stats=COURSE_STATS (el de train)
    X_split, _ = build_transformer_block_for_split(
        split_students_al=df_stud,
        split_interactions=df_inter,
        weeks=W,
        course_stats=COURSE_STATS,  # <--- CLAVE: Usar stats de train
        p_clip=0.995,
    )
    
    # Alinear target
    y_split = df_tgt.loc[X_split.index].copy()
    
    # Checks básicos
    print(f"   -> Shape X: {X_split.shape}, y: {y_split.shape}")
    if X_split.isna().sum().sum() > 0:
        print(f"   ⚠️ ALERTA: NaNs detectados en {split_name}!")
        
    # Guardado
    out_dir = OUT_DIR / split_name
    out_dir.mkdir(parents=True, exist_ok=True)
    
    X_split.to_csv(out_dir / f"w{W:02d}_X_seq.csv")
    y_split.to_csv(out_dir / f"w{W:02d}_y.csv")
    
    print(f"   💾 Guardado en: {out_dir}")

print("\n✅ Todos los splits generados para ventana W =", W)

NameError: name 'val_students_al' is not defined

In [ ]:
# =========================
# [CELDA INDEPENDIENTE - AUTÓNOMA] Estáticas desde students.csv + schema TRAIN + scaler + merge con FE2 + guardado
# =========================
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# ---- CONFIG ----
W = 12  # cambia a 16/20/24 si quieres

# Ruta raíz del repo (ajústala si tu notebook no está en /workspace/TFM_education_ai_analytics)
PROJECT_DIR = "/workspace/TFM_education_ai_analytics"

PROCESSED_DIR = os.path.join(PROJECT_DIR, "data/2_processed")
FE2_DIR = os.path.join(PROJECT_DIR, "data/6_transformer_features")

SPLITS = ["training", "validation", "test"]

STATIC_CATEGORICAL = ["gender", "region", "highest_education", "imd_band", "age_band", "disability"]
STATIC_NUMERIC = ["num_of_prev_attempts", "studied_credits", "date_registration", "module_presentation_length"]
EXCLUDE_COLS = ["final_result", "date_unregistration"]  # leakage / target-like

# ---- HELPERS ----
def build_unique_id(df_students: pd.DataFrame) -> pd.Series:
    return (
        df_students["id_student"].astype(str)
        + "_"
        + df_students["code_module"].astype(str)
        + "_"
        + df_students["code_presentation"].astype(str)
    )

def load_students(split: str) -> pd.DataFrame:
    path = os.path.join(PROCESSED_DIR, split, "students.csv")
    if not os.path.exists(path):
        raise FileNotFoundError(f"No existe: {path}")
    df = pd.read_csv(path)
    return df

def load_fe2_seq(split: str) -> pd.DataFrame:
    path = os.path.join(FE2_DIR, split, f"w{W:02d}_X_seq.csv")
    if not os.path.exists(path):
        raise FileNotFoundError(f"No existe FE2: {path}")
    return pd.read_csv(path, index_col=0)

def build_static_from_students(df_students: pd.DataFrame) -> pd.DataFrame:
    df = df_students.copy()

    # Excluir columnas peligrosas
    for c in EXCLUDE_COLS:
        if c in df.columns:
            df = df.drop(columns=[c])

    # unique_id
    df["unique_id"] = build_unique_id(df)
    df = df.set_index("unique_id")

    # Numéricas
    num_cols = [c for c in STATIC_NUMERIC if c in df.columns]
    for c in num_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0.0).astype(float)

    # Categóricas → OHE
    cat_cols = [c for c in STATIC_CATEGORICAL if c in df.columns]
    X_cat = pd.get_dummies(df[cat_cols].fillna("UNKNOWN").astype(str), prefix=cat_cols)

    X_num = df[num_cols].copy()
    X = pd.concat([X_num, X_cat], axis=1)

    X = X.replace([np.inf, -np.inf], np.nan).fillna(0.0)
    X.index.name = "unique_id"
    return X

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

# ---- LOAD DATA ----
students = {split: load_students(split) for split in SPLITS}
X_seq = {split: load_fe2_seq(split) for split in SPLITS}

print("✅ loaded students:", {k: v.shape for k, v in students.items()})
print("✅ loaded FE2 seq :", {k: v.shape for k, v in X_seq.items()})

# ---- BUILD STATIC (TRAIN defines schema + scaler) ----
X_static_train = build_static_from_students(students["training"])
static_schema = X_static_train.columns.tolist()

num_cols_in_schema = [c for c in STATIC_NUMERIC if c in X_static_train.columns]

scaler_static = StandardScaler()
if num_cols_in_schema:
    X_static_train[num_cols_in_schema] = scaler_static.fit_transform(X_static_train[num_cols_in_schema])

def build_static_split(split: str) -> pd.DataFrame:
    Xs = build_static_from_students(students[split])
    Xs = Xs.reindex(columns=static_schema, fill_value=0.0)
    cols = [c for c in num_cols_in_schema if c in Xs.columns]
    if cols:
        Xs[cols] = scaler_static.transform(Xs[cols])
    return Xs.replace([np.inf, -np.inf], np.nan).fillna(0.0)

X_static = {
    "training": X_static_train,
    "validation": build_static_split("validation"),
    "test": build_static_split("test"),
}

print("✅ static shapes:", {k: v.shape for k, v in X_static.items()})

# ---- MERGE STATIC + FE2 ----
X_hybrid = {}
for split in SPLITS:
    X_hybrid[split] = X_seq[split].join(X_static[split], how="left").fillna(0.0)
    print(f"✅ hybrid {split}: {X_hybrid[split].shape}")

# ---- SAVE ----
for split in SPLITS:
    out_dir = os.path.join(FE2_DIR, split)
    ensure_dir(out_dir)
    out_path = os.path.join(out_dir, f"w{W:02d}_X_hybrid.csv")
    X_hybrid[split].to_csv(out_path)
    print("💾 guardado:", out_path)

# Preview
display(X_hybrid["training"].head(3))


FileNotFoundError: No existe FE2: /workspace/TFM_education_ai_analytics/data/6_transformer_features/validation/w12_X_seq.csv